## Building a BATTLE Eval

In [15]:
file_name = "dataset"
model = "gpt-3.5-turbo"
choices = "user"                     # options: user, ai
num_few_shot = 0
initial_message = [{"role": "user", "content": ""}]
roles = "user"                      # options: system-user, system, user
battles_generation = "complete-chat"        # options: dense, only-new, complete-chat

#!cd evals
#!git lfs fetch --all
#!git lfs pull

In [16]:
import os
from openai import OpenAI
import pandas as pd
import yaml
from IPython.display import display_markdown
import copy

client = OpenAI()
#import datetime
#from dotenv import load_dotenv

#load_dotenv()  # Load environment variables from .env file

#api_key = os.environ.get("OPENAI_API_KEY")

# Install Evals if you haven't already
# %pip install -e ../.
# pip install --upgrade openai
# %pip install pandas


In [17]:
# Paths. Assuming this notebook is in examples/

evals_path = os.path.join(os.getcwd(), "..", "evals")

registry_path = os.path.join(evals_path, "registry", "evals", "battles.yaml")

data_path = os.path.join(evals_path, "registry", "data", "battles")
os.makedirs(data_path, exist_ok=True)
data_path = os.path.join(data_path, "samples.jsonl")

json_logs_path = os.path.join(os.getcwd(), "logs")
os.makedirs(json_logs_path, exist_ok=True)
json_logs_path = os.path.join(json_logs_path, "logs")

df_path = os.path.join(evals_path, "evallogs", "df")
os.makedirs(df_path, exist_ok=True)
dataset_path = os.path.join(df_path, file_name)

In [18]:
# Registry yaml

registry_yaml = {}

registry_yaml["battles"] = {
    "id": "battles.test.v1",
    "metrics": ["accuracy"]
}
registry_yaml["battles.test.v1"] = {
    "class": "evals.elsuite.modelgraded.classify:ModelBasedClassify",
    "args": {
        "samples_jsonl": "battles/samples.jsonl",
        "eval_type": "cot_classify",
        "modelgraded_spec": "battle"
    }
}
with open(os.path.join(registry_path), "w") as f:
    yaml.dump(registry_yaml, f)

In [19]:
def best(dataset, memory_limit=False):
    #best_message = dataset[dataset["Choice"]=="Yes"].iloc[-1]["Instruction1"]
    #best_response = dataset[dataset["Choice"]=="Yes"].iloc[-1]["Response1"]
    if dataset.iloc[-1]["Choice"] == "Yes":
        best_content = dataset.iloc[-1]["Instruction1"]
        best_response = dataset.iloc[-1]["Response1"]
    else:
        best_content = dataset.iloc[-1]["Instruction2"]
        best_response = dataset.iloc[-1]["Response2"]
    if memory_limit:
        best_message = [{"role": "user", "content": best_content}]
    if not memory_limit: 
        try:
            best_message = dataset[dataset["Choice"]=="Yes"].iloc[-1]["Data"]["Full message"]
        except IndexError:
            best_message = dataset.iloc[0]["Data"]["Full message"]
    return best_message,best_response

In [20]:
# Load data

try:
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    best_message, best_response = best(dataset)

# Or create new data file

except FileNotFoundError:
    completion = client.chat.completions.create(
        messages = initial_message,
        model = model,
        )
    dataset = pd.DataFrame(
        [
            ["", "", "", "", "", "", {}],
            ["", completion.choices[0].message.content, "", "", "", "", {}]
            ], 
        columns=['Instruction1', 'Response1', 'Instruction2', 'Response2', 'Sampled', 'Choice', 'Data']
    ) 
    dataset.at[0, "Data"] = {"Completion":completion.to_dict(),"Full message":initial_message}
candidate_messages = [initial_message]
next_candidate_messages = []
generation_distance = 0
message_in_candidate_messages = 0
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")


In [21]:
def battle(best_message, best_response, candidate_message):

    dataset = [{"input1": candidate_message, "input2": best_message, "completion2":best_response}]

    df = pd.DataFrame(dataset)
    df.to_json(data_path, orient="records", lines=True)

    !oaieval gpt-3.5-turbo battles --record_path logs/logs
    
    with open(json_logs_path, "r") as f:
        df = pd.read_json(f, lines=True)

    #current_time = datetime.datetime.now()
    #formatted_time = current_time.strftime("%Y%m%d_%H%M%S")
    #df.to_json(os.path.join(df_path, formatted_time), lines=True, orient="records")

    instruction1 = candidate_message[-1]["content"]
    instruction2 = best_message[-1]["content"]

    battle_prompt_content = df["data"].iloc[-2]["prompt"][0]["content"]
    response1 = battle_prompt_content.split("\n[Response 1]\n",)[1].split("\n\n[Instruction 2]\n")[0]
    response1 = response1.replace("\\'", "'").replace("\\n", "\n")
    response2 = battle_prompt_content.split("\n[Response 2]\n",)[1].split("\n\n\nIs the first response better than the second?")[0]
    response2 = response2.replace("\\'", "'").replace("\\n", "\n")
    print(f"Response1: {response1}")
    #print(f"response2: {response2}")

    sampled = df["data"].iloc[-2]["sampled"][0]

    choice = df["data"].iloc[-1]["choice"]

    data = {'Instruction1': instruction1, 'Response1': response1, 'Instruction2': instruction2, 'Response2': response2, 'Sampled': sampled, 'Choice': choice, 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = df.to_dict()
    return data

In [22]:
def switch_system_and_user(message):
    new_message = []
    if message[0]["role"] == "system":
        new_message.append([{"role": "user", "content": message[0]["content"]}])
    else:
        new_message.append([{"role": "system", "content": message[0]["content"]}])
    return new_message

In [23]:
def list_candidate_messages(dataset,generation_distance,roles="user"):
    best_message = best(dataset)[0]
    best_content = best_message[-1]["content"]
    last = [best_content]
    list_of_contents = [best_content]

    next = []
    for i in range(generation_distance):
        #print (f"starting level {i}, generation distance {generation_distance}")
        for content in last:
            #print(f"last_up: {last_up}, now {content}")
            next.extend(find_parents(content, dataset))
            next.extend(find_children(content, dataset))
            
        #print (f"behind for loops")
        list_of_contents.extend(next)
        last = next.copy()
        #print(f"endind level {i}")
        
    list_of_contents = pd.array(list_of_contents).unique().tolist()
    
    #best_message_index = list_of_contents.index(best_message[0][content])
    #start = max(best_message_index - generation_distance, 0)
    #stop = min(len(list_of_contents))
    #stop = len(list_of_contents) - 1

    messages = []
    for content in list_of_contents:
        if roles == "system-user" or "user":
            messages.append([{"role":"user","content":content}])
        if (roles == "system-user") or (roles == "system"):
            print("Creating message without user role.")
            messages.append([{"role":"system","content":content}])

    return messages

In [24]:
def make_x_shot_prompt(dataset,message,num_few_shot):
    few_shot_prompt = []
    for _, key_battle in dataset[dataset["Choice"]=="Yes"].iloc[-num_few_shot:].iterrows():
        few_shot_prompt.append({"role": "system", "content": key_battle["Instruction1"], "name": "example_user"})
        few_shot_prompt.append({"role": "system", "content": key_battle["Response1"], "name": "example_assistant"})
    message = few_shot_prompt + message
    return message

In [25]:
def transform_roles(chat,assistant_transform_type = "user-assistant"):
    prompt_messages = copy.deepcopy(chat)
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type
    return prompt_messages

In [26]:
def user_choice_prompt(candidate_message,n,best_message,best_response):

    prompt = "You are comparing two responses to the following two instructions."

    prompt += "\n\n[Instruction 1]\n"
    prompt += candidate_message[-1]["content"]

    prompt += "\n\n[Response 1]\n"
    completion = client.chat.completions.create(
        messages = candidate_message,
        model = model,
        )
    response1 = completion.choices[0].message.content
    prompt += response1
    
    prompt += "\n\n[Instruction 2]\n"
    best_content = best_message[-1]["content"]
    prompt += best_content

    prompt += "\n\n[Response 2]\n"
    prompt += best_response

    prompt += "\n\n\nIs the first response better than the second? You must provide one answer based on your subjective view."

    data = {'Instruction1': candidate_message[-1]["content"], 'Response1': response1, 'Instruction2': best_content, 'Response2': best_response, 'Sampled': "", 'Choice': "", 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict(),"Full message":candidate_message,"Parent":n}

    display_markdown(prompt, raw=True)
    return data,candidate_messages,message_in_candidate_messages,generation_distance

In [27]:
def user_choice_prompt_old(candidate_messages,message_in_candidate_messages,dataset, generation_distance, roles, battles_generation,num_few_shot):

    prompt = "You are comparing two responses to the following two instructions."

    prompt += "\n\n[Instruction 1]\n"
    if battles_generation == "dense":
        if message_in_candidate_messages < len(candidate_messages):
            candidate_message = candidate_messages[message_in_candidate_messages]
            message_in_candidate_messages += 1
        else:
            candidate_messages = list_candidate_messages(dataset, generation_distance, roles)
            generation_distance += 1
            candidate_message = candidate_messages[0]
            message_in_candidate_messages = 1
            print(f"candidate messages listed, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")
    if battles_generation == "complete-chat":
        chat = best(dataset)[0] + [{'role': 'assistant', 'content': best(dataset)[1]}]
        candidate_message = transform_roles(chat)
    if num_few_shot > 0:
        candidate_message = make_x_shot_prompt(dataset,candidate_message,num_few_shot)
    prompt += candidate_message[-1]["content"]

    prompt += "\n\n[Response 1]\n"
    completion = client.chat.completions.create(
        messages = candidate_message,
        model = model,
        )
    response1 = completion.choices[0].message.content
    prompt += response1
    
    prompt += "\n\n[Instruction 2]\n"
    best_content = best(dataset)[0][-1]["content"]
    prompt += best_content

    prompt += "\n\n[Response 2]\n"
    prompt += best(dataset)[1]

    prompt += "\n\n\nIs the first response better than the second? You must provide one answer based on your subjective view."

    data = {'Instruction1': candidate_message[-1]["content"], 'Response1': response1, 'Instruction2': best_content, 'Response2': best(dataset)[1], 'Sampled': "", 'Choice': "", 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict(),"Full message":candidate_message}

    display_markdown(prompt, raw=True)
    return data,candidate_messages,message_in_candidate_messages,generation_distance

In [28]:
"""if choices == "user":
    data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt_old(
        candidate_messages,
        message_in_candidate_messages,
        dataset,
        generation_distance,
        roles,
        battles_generation,
        num_few_shot)"""

'if choices == "user":\n    data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt_old(\n        candidate_messages,\n        message_in_candidate_messages,\n        dataset,\n        generation_distance,\n        roles,\n        battles_generation,\n        num_few_shot)'

In [29]:
def continue_chat(n, dataset_path):
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    last_response = dataset.iloc[n]["Response1"]
    chat = dataset.iloc[n]["Data"]["Full message"] + [{'role': 'assistant', 'content': last_response}]
    candidate_message = transform_roles(chat)
    last_response = dataset.iloc[n]["Response1"]
    parent = n
    data = user_choice_prompt(candidate_message,parent,candidate_message,last_response)[0]
    return data

In [30]:
def improve_chat(n, dataset_path):
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    candidate_message = dataset.iloc[n]["Data"]["Full message"]
    last_response = dataset.iloc[n]["Response1"]
    parent = dataset.iloc[n]["Data"]["Parent"]
    data = user_choice_prompt(candidate_message,parent,candidate_message,last_response)[0]
    return data

In [ ]:
dataset.iloc[-1:]

In [ ]:
n = 10
data = improve_chat(n,dataset_path)

In [ ]:
choice = "No"
choice = "Yes"
data.at[0, "Choice"] = choice
print(f"Your answer:{choice}")
if choice == "Yes":
    print(f"New best message and response!")
    if battles_generation == "dense":
        generation_distance = 0
dataset = pd.concat([dataset, data],ignore_index=True)
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
#data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt_old(candidate_messages,message_in_candidate_messages,dataset,generation_distance,roles,battles_generation,num_few_shot)

In [ ]:
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")


In [ ]:
def is_parent(parent,response,child,):
    if parent["content"] == "":
        parent = []
    lp = len(parent)
    lch = len(child)
    if lp != lch + 1:
        return False
    if msg_to_content(child)[:l+1] != msg_to_content(parent).append(response):
        return False
    else:
        return True

In [ ]:
def msg_to_content(msg):
    content = []
    for line in msg:
        content.append(line["content"])
    return content

In [ ]:
def best_msg_level_up(dataset, msg, best_chat):
    level = 0 if best_chat.empty else len(msg)
    filtered_dataset = dataset[
        (dataset["Choice"] != "No") & 
        (dataset["Data"].apply(lambda x: len(x["Full message"])) == level + 1) &
        (dataset["Data"].apply(lambda x: is_parent(x["Full message"], msg, dataset)))
        ] 
    if len(filtered_dataset) > 1:
        #print(filtered_dataset)
        filtered_dataset = filtered_dataset.iloc[[-1]]
        print(filtered_dataset)
    if filtered_dataset.empty:
        return best_chat
    best_chat = pd.concat([best_chat, filtered_dataset],ignore_index=True)
    #try:
    #print(filtered_dataset.iloc[0])
    msg = filtered_dataset.iloc[0]["Data"]["Full message"]
    #except:
        #print("Returning filtered dataset")
        #return filtered_dataset.iloc[0]
    best_msg_level_up(dataset, msg, best_chat)
    return best_chat
    

In [ ]:
best_chat = best_msg_level_up(dataset, initial_message, pd.DataFrame([],columns=dataset.columns))

In [ ]:
dataset.filter([1,3],axis=0)

In [34]:
def parent(n,dataset=dataset):
    if n == 0:
        return None
    return dataset.iloc[n]["Data"]["Parent"]

In [35]:
def children(n, dataset=dataset):
    filtered_dataset = dataset[dataset["Data"].apply(lambda x: x["Parent"] == n)]
    return filtered_dataset.index.to_list()

In [79]:
def ancestors(n,dataset=dataset):
    ancestors = []
    while n > 0:
        ancestors.append(parent(n))
        n = parent(n)
    ancestors.reverse()
    return ancestors

In [37]:
def descendants(n,dataset=dataset):
    def descendants_recursion(n,l,dataset=dataset):
        l.extend(children(n))
        for child in children(n):
            descendants_recursion(child,l)
        return l
    l = []
    return descendants_recursion(n,l)

In [ ]:
if choices == "ai":
    for candidate_message in candidate_messages:
        if num_few_shot > 0:
            candidate_message = make_x_shot_prompt(dataset,candidate_message,num_few_shot)
        data = battle(best_message, best_response, candidate_message)
        dataset = pd.concat([dataset, data],ignore_index=True)
        dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
        if data["Choice"].iloc[0] == "Yes":
            best_message = candidate_message
            best_response = data["Response1"].iloc[0]
            print(f"New best message:{best_message}")
            if battles_generation == "dense":
                generation_distance = 0

        print(f"Instruction1: {candidate_message[-1]["content"]}")

        if battles_generation == "only-new":
            new_message = [{"role": candidate_message[0]["role"], "content":data["Response1"].iloc[0]}]
            if new_message not in next_candidate_messages:
                next_candidate_messages.append(new_message)
                if roles == "system-user":
                    next_candidate_messages.append(switch_system_and_user(new_message))

    if battles_generation == "only-new":
        candidate_messages = next_candidate_messages
        next_candidate_messages = []
    if battles_generation == "dense":
        #print("going to list candidate messages")
        candidate_messages = list_candidate_messages(dataset, best_message, generation_distance, roles)
        generation_distance += 1
        #print("candidate messages listed")
    print(f"all done, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")

In [31]:
filtered_dataset = dataset[(dataset["Choice"] != "No")]
best_responses = set(filtered_dataset.index)

In [125]:
def display_chat(chat):
    for row in dataset.filter(chat,axis=0).iterrows():
        display_markdown(row[0],raw=True)
        display_markdown(row[1].iloc[1],raw=True)

In [129]:
display_chat(best_chats(dataset)[0])

1

Hello! How can I assist you today?

2

Hello! I'm here to help you with any questions you might have or provide information on a variety of topics. Feel free to ask me anything!

3

...

I'm delighted to hear your enthusiasm for exploring specific use cases and real-world examples of AI applications in healthcare! By examining these instances, we can gain valuable insights into the practical impact of AI technologies in revolutionizing patient care, enhancing clinical decision-making, and driving innovation in the medical field.

AI-driven solutions for medical image analysis, predictive diagnostics, personalized treatment recommendations, and other healthcare applications showcase the transformative power of AI in enhancing healthcare delivery and improving patient outcomes. By diving into these case studies and examples, we can better understand the capabilities of AI in addressing key challenges in healthcare and accelerating advancements in the field.

If you would like to explore specific AI-driven use cases or examples in healthcare, such as successful implementations of AI for disease detection, clinical decision support, or health monitoring, please feel free to indicate your preferences. Additionally, if you are interested in delving into the technical aspects of AI algorithms and models used in healthcare scenarios, such as deep learning for medical imaging analysis or natural language processing for clinical data extraction, we can explore these topics in further detail.

Together, we can uncover the innovative solutions and advancements that are transforming the future of healthcare through AI. I am here to provide detailed information, analyze notable use cases, and engage in meaningful discussions to deepen our understanding of AI applications in healthcare. Let's continue our exploration and delve into the transformative potential of AI in shaping the future of medicine.

19

I share your excitement for exploring the transformative potential of AI in healthcare through specific use cases, real-world examples, and in-depth discussions on the practical applications of AI technologies in improving patient care, diagnosis, treatment, and overall healthcare outcomes.

Delving into successful implementations of AI for disease detection, clinical decision support, personalized treatment recommendations, and other healthcare applications will allow us to gain insights into the ways AI is revolutionizing healthcare delivery and reshaping the patient experience. By examining these use cases, we can better understand the impact of AI in addressing complex healthcare challenges, driving medical innovation, and enhancing the quality of care provided to patients.

If you have a specific area of interest within AI applications in healthcare or a particular use case you would like to explore further, please feel free to share your preferences. Whether you are interested in exploring AI-driven solutions for medical imaging analysis, predictive diagnostics, patient monitoring, or AI-powered virtual health assistants, I am here to provide detailed information, analyze case studies, and engage in insightful discussions on these topics.

Together, we can continue our exploration of AI in healthcare, uncovering the innovative solutions and advancements that are shaping the future of medicine through the integration of AI technologies. I look forward to furthering our conversation and delving into the transformative potential of AI in driving healthcare innovation and improving patient outcomes. Let's embark on this journey together and discover the profound impact of AI in healthcare.

20

I'm thrilled to join you on this exploration of the transformative potential of AI in healthcare! By delving into specific use cases, real-world examples, and in-depth discussions on the practical applications of AI technologies in improving patient care, diagnosis, treatment, and healthcare outcomes, we can gain a comprehensive understanding of how AI is reshaping the landscape of medical practices and patient experiences.

With a focus on successful implementations of AI in disease detection, clinical decision support, personalized treatment recommendations, and other healthcare applications, we can uncover the innovative solutions that are revolutionizing the way healthcare is delivered and experienced. These use cases exemplify how AI is being leveraged to address complex healthcare challenges, enhance medical decision-making, and optimize patient care pathways, ultimately leading to improved health outcomes and enhanced quality of care for patients.

Whether you are interested in exploring AI-driven solutions for medical imaging analysis, predictive diagnostics, patient monitoring, or virtual health assistants, I am here to provide detailed insights, analyze case studies, and engage in insightful discussions to deepen our understanding of AI applications in healthcare. Together, we can continue to explore the boundless possibilities and advancements that AI brings to the realm of healthcare, driving innovation, and transforming patient-centered care.

I look forward to embarking on this journey with you and unraveling the profound impact of AI in healthcare. Let's continue our exploration and discover the remarkable potential of AI in shaping the future of medicine and healthcare delivery. If there are specific areas of interest or use cases you would like to delve into further, please feel free to share your preferences, and we can explore them together.

In [98]:
for row in dataset.filter(ancestors(20),axis=0).iterrows():
    display_markdown(row[0],raw=True)
    display_markdown(row[1].iloc[1],raw=True)


1

Hello! How can I assist you today?

2

Hello! I'm here to help you with any questions you might have or provide information on a variety of topics. Feel free to ask me anything!

3

...

I share your enthusiasm for exploring specific use cases and real-world examples of AI applications in healthcare, as these instances offer valuable insights into the practical impact of AI technologies in transforming patient care and medical practices. By delving into these case studies and examples, we can gain a deeper understanding of how AI is revolutionizing healthcare delivery, improving clinical outcomes, and driving innovation in the medical field.

Whether you are interested in learning about AI-driven solutions for medical image analysis, predictive diagnostics, personalized treatment recommendations, or other healthcare applications, I am here to provide detailed information, analyze noteworthy use cases, and engage in insightful discussions on these topics.

Furthermore, if you wish to explore the technical aspects of AI algorithms and models used in specific healthcare scenarios, such as machine learning algorithms for disease detection, natural language processing for clinical documentation, or reinforcement learning for treatment optimization, I would be happy to delve into the intricacies of these AI technologies with you.

Please feel free to specify any particular use cases or examples of AI in healthcare that you would like to explore further, or if you have any specific areas of interest within the realm of AI technology and its applications in the medical field. I am here to assist you in uncovering the innovative solutions and advancements that are reshaping the future of healthcare through AI. Let's continue our exploration together and delve deeper into the transformative potential of AI in healthcare.

18

I'm delighted to hear your enthusiasm for exploring specific use cases and real-world examples of AI applications in healthcare! By examining these instances, we can gain valuable insights into the practical impact of AI technologies in revolutionizing patient care, enhancing clinical decision-making, and driving innovation in the medical field.

AI-driven solutions for medical image analysis, predictive diagnostics, personalized treatment recommendations, and other healthcare applications showcase the transformative power of AI in enhancing healthcare delivery and improving patient outcomes. By diving into these case studies and examples, we can better understand the capabilities of AI in addressing key challenges in healthcare and accelerating advancements in the field.

If you would like to explore specific AI-driven use cases or examples in healthcare, such as successful implementations of AI for disease detection, clinical decision support, or health monitoring, please feel free to indicate your preferences. Additionally, if you are interested in delving into the technical aspects of AI algorithms and models used in healthcare scenarios, such as deep learning for medical imaging analysis or natural language processing for clinical data extraction, we can explore these topics in further detail.

Together, we can uncover the innovative solutions and advancements that are transforming the future of healthcare through AI. I am here to provide detailed information, analyze notable use cases, and engage in meaningful discussions to deepen our understanding of AI applications in healthcare. Let's continue our exploration and delve into the transformative potential of AI in shaping the future of medicine.

19

I share your excitement for exploring the transformative potential of AI in healthcare through specific use cases, real-world examples, and in-depth discussions on the practical applications of AI technologies in improving patient care, diagnosis, treatment, and overall healthcare outcomes.

Delving into successful implementations of AI for disease detection, clinical decision support, personalized treatment recommendations, and other healthcare applications will allow us to gain insights into the ways AI is revolutionizing healthcare delivery and reshaping the patient experience. By examining these use cases, we can better understand the impact of AI in addressing complex healthcare challenges, driving medical innovation, and enhancing the quality of care provided to patients.

If you have a specific area of interest within AI applications in healthcare or a particular use case you would like to explore further, please feel free to share your preferences. Whether you are interested in exploring AI-driven solutions for medical imaging analysis, predictive diagnostics, patient monitoring, or AI-powered virtual health assistants, I am here to provide detailed information, analyze case studies, and engage in insightful discussions on these topics.

Together, we can continue our exploration of AI in healthcare, uncovering the innovative solutions and advancements that are shaping the future of medicine through the integration of AI technologies. I look forward to furthering our conversation and delving into the transformative potential of AI in driving healthcare innovation and improving patient outcomes. Let's embark on this journey together and discover the profound impact of AI in healthcare.

In [123]:
bch = best_chats(dataset)

In [124]:
bch

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 [0, 1, 22],
 [0, 1, 2, 23],
 [0, 1, 2, 3, 4, 25],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 29],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 30]]

In [128]:
def best_chats(dataset):
    best_chats = []
    best_responses = set(dataset[(dataset["Choice"] != "No")].index)
    ancestor_responses = set()
    for n in best_responses:
        ancestor_responses = ancestor_responses.union(ancestors(n))
    best_chats_ends = best_responses.difference(ancestor_responses)
    for chat_end in best_chats_ends:
        chat = ancestors(chat_end)
        chat.append(chat_end)
        best_chats.append(chat)
    return best_chats
    
                                
    #(dataset["Data"].apply(lambda x: len(x["Full message"])) == level + 1) &
    #(dataset.apply(lambda x: is_parent(msg,x["Data"]["Full message"],x["Response1"])))


In [ ]:
for entry in dataset["Response1"]:
    display_markdown(entry, raw=True)

In [ ]:
test = dataset.iloc[30]
for text in test:
        display_markdown(text, raw=True)

In [ ]:
dataset.loc[30,"Choice"] = "Maybe"

In [ ]:
test = dataset[dataset["Choice"]=="Yes"]#.iloc[0]
for row in test.iterrows():
        display_markdown(row[0], raw=True)
        display_markdown(row[1].iloc[1], raw=True)
